### Introduction: Running OpenMM simulations
OpenMM is a bit different from other MD simulation programs because there is no executable OpenMM program; instead it has an easy to use Python API for setting up and running simulations.  One could think of the OpenMM input file as being a short Python script or Jupyter notebook.

#### Resources
The [OpenMM Script Builder](http://builder.openmm.org/) is a web app that assists in creating Python scripts for OpenMM simulations.  Once you are done with this notebook and ready to use features like protein force fields, periodic boundary conditions and temperature / pressure controls, you may use the Builder to generate a script with the correct OpenMM API calls.

The [OpenMM documentation page](http://openmm.org/documentation.html) is very high-quality. Before reading the docs you're encouraged to watch the two short videos on the landing page given by Peter Eastman, the OpenMM developer.  There are also various OpenMM-related videos that you can find on YouTube from past workshops given at Stanford, including some from Lee-Ping.

In [ ]:
%matplotlib inline
import nglview
import numpy as np
import matplotlib.pyplot as plt
# OpenMM imports
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
# Import the sys module because sys.stdout is the file object
# that represents printing to the console.
import sys

#### Create a `PDBFile` object by passing a `.pdb` file.
This object will contain the atomic positions in `pdb.positions` and the topology object in `pdb.topology`. 

This `.pdb` file contains 27 xenon atoms arranged in a cube; it's more of a demo-sized simulation than a research-sized one.

In [ ]:
# Step 1: Create PDB object that contains positions and topology.
pdb = PDBFile('xenon.pdb')

#### Create a `ForceField` object by passing a force field `.xml` file.
This object represents the force field definition, including parameters and residue templates. It will contain the method `forcefield.createSystem(topology)` for creating the `System` object given `pdb.topology`.

It is possible to read and understand very simple force field `.xml` files so you are encouraged to open the files in the text editor to see what information it contains.

In [ ]:
forcefield = ForceField('xenon.xml')

#### Create a `System` object.
This object represents the parameterized system containing all particles and their individual interactions. In our case, the only interactions are the Lennard-Jones interactions between each pair of xenon atoms.

In [ ]:
system = forcefield.createSystem(pdb.topology)

#### Create an `Integrator` object.

This object represents the algorithm for evolving the system in time. The `VerletIntegrator` uses a leapfrog Verlet algorithm that is similar (but not identical) to the velocity Verlet algorithm covered in lecture.

Note the use of `picosecond`: this is an OpenMM unit object that we got from importing `simtk.unit` into the global namespace.

In [ ]:
integrator = VerletIntegrator(0.001*picosecond)

#### Create a `Platform` object.
This object represents which set of low-level codes are being used to actually evaluate the interactions. OpenMM contains a simple `Reference` platform, an optimized `CPU` platform, an optimized `CUDA` platform for NVidia GPUs, and an optimized `OpenCL` platform for AMD GPUs. If a `Platform` is not explicitly created and provided to the creation of the `Simulation` object, a default will be used.

Because this system is so small, the `Reference` platform in fact gives the best performance, so that's what we'll use here.

In [ ]:
platform = Platform.getPlatformByName('Reference')

#### Create a `Simulation` object.
This object is the central one that you interact with when running simulations. It is created by passing in the various objects that you created above, and it contains methods for carrying out basic simulation tasks (see diagram below for how various classes are related).

In [ ]:
simulation = Simulation(pdb.topology, system, integrator, platform)

![OpenMM diagram](https://i.imgur.com/BI6M5pp.png)

#### Set the initial positions.
Up to this point we haven't done anything with the positions.  The positions from the `PDBFile` must be explicitly passed to `Simulation` before any actual simulating can take place.

Note that the method is actually contained in `simulation.context`.  The `Context` object contains most of the functions of `Simulation`; technically speaking  `Context` is a C++ API layer object, and `Simulation` is defined in the Python layer and adds a few convenience functions.

In [ ]:
simulation.context.setPositions(pdb.positions)

#### Compute the energy.
To compute the energy in OpenMM, we use `simulation.context.getState()`. For performance reasons, this function only retrieves properties that the user asks for, so you should look at the API for how to retrieve other properties such as the positions and velocities.

In [ ]:
state = simulation.context.getState(getEnergy=True)
print(state.getPotentialEnergy())

#### Create Reporter objects and add them to the Simulation.
Reporter objects periodically return data as the simulation is running. `StateDataReporter` prints current values of variables to a file or the console, and `PDBReporter` writes the current positions to a trajectory `.pdb` file.

Reporter objects must be added to the list `simulation.reporters` after being created.

In [ ]:
reporter1 = StateDataReporter(sys.stdout, 100, step=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True)
reporter2 = PDBReporter('xenon-output.pdb', 100)
simulation.reporters.append(reporter1)
simulation.reporters.append(reporter2)

#### Run the MD simulation.
Running the simulation itself is as easy as calling `Simulation.step(number_of_steps)`. Here, one million steps are carried out using a time step of 0.001 ps (defined above) to give a total of 1 ns of simulation time.  

Once the simulation is started by calling this method, it will print information to the console regarding current values of the simulation time, potential / kinetic / total energy, and a .pdb file will be written to the current folder that you will be able to view in VMD.

It is also possible to use the `nglview` package to view trajectories but the `pytraj` package may be required.

In [ ]:
simulation.step(1000000)